In [1]:
import pandas as pd
import os
os.chdir('/Users/Ali/Desktop/Mehtonen et al./GSE148218_RAW')
files = [f for f in os.listdir() if "REH" in f]
files

['GSM4795825_REH_TK', 'GSM4795824_REH_DMSO', 'GSM4795826_REH_XR']

In [5]:
from pathlib import Path
folder = Path(files[0])
print(folder)
barcode_file = folder / "barcodes.tsv.gz" # str(folder + "/barcodes.tsv.gz")
print(barcode_file)

GSM4795825_REH_TK
GSM4795825_REH_TK/barcodes.tsv.gz


In [26]:
bc = pd.read_csv(barcode_file, sep="\t")
bc.drop(["hg19_counts", "mm10_counts", "call"], axis=1, inplace=True)
bc

KeyError: "['hg19_counts', 'mm10_counts', 'call'] not found in axis"

In [ ]:
bc.to_csv(barcode_file, compression='gzip', header=False, index=False)

In [6]:
folder = Path(files[0])
print(folder)
features_file = folder / "features.tsv.gz"
print(features_file)
fc = pd.read_csv(features_file, sep="\t")
fc

GSM4795825_REH_TK
GSM4795825_REH_TK/features.tsv.gz


,Unnamed: 0,gene_ids,feature_types,genome
0,DDX11L1,hg19_ENSG00000223972,Gene Expression,hg19
1,WASH7P,hg19_ENSG00000227232,Gene Expression,hg19
2,MIR1302-10,hg19_ENSG00000243485,Gene Expression,hg19
3,FAM138A,hg19_ENSG00000237613,Gene Expression,hg19
4,OR4G4P,hg19_ENSG00000268020,Gene Expression,hg19
...,...,...,...,...
57900,CU442762.4,hg19_ENSG00000271726,Gene Expression,hg19
57901,AC002321.2,hg19_ENSG00000215616,Gene Expression,hg19
57902,AC002321.1,hg19_ENSG00000215611,Gene Expression,hg19
57903,CT867977.1,hg19_ENSG00000265557,Gene Expression,hg19


In [ ]:
folder = Path(files[0])
print(folder)
matrix_file = folder / "matrix.mtx.gz"
print(matrix_file)
mx = pd.read_csv(matrix_file, sep="\t")
mx

GSM4795825_REH_TK
GSM4795825_REH_TK/matrix.mtx.gz


,%%MatrixMarket matrix coordinate real general
0,%
1,2144 57905 7495000
2,1 75 1.0000000e+00
3,1 106 1.0000000e+00
4,1 110 1.0000000e+00
...,...
7494997,2144 57759 6.0000000e+00
7494998,2144 57761 2.0000000e+00
7494999,2144 57771 3.0000000e+00
7495000,2144 57846 5.0000000e+00


In [27]:
import scanpy as sc

adata=sc.read_h5ad(str(folder))
adata

IsADirectoryError: [Errno 21] Unable to open file (file read failed: time = Thu Jul 20 10:33:07 2023
, filename = 'GSM4795825_REH_TK', file descriptor = 71, errno = 21, error message = 'Is a directory', buf = 0x7ff7b134d588, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)

In [18]:
os.listdir(folder)

['.DS_Store', 'features.tsv.gz', 'barcodes.tsv.gz', 'matrix.mtx.gz']

In [20]:
from scipy.io import mmread
A= mmread(Path(folder) / "matrix.mtx.gz")

In [21]:
from scipy.sparse import csr_matrix

In [32]:
csr_matrix(A)

<2144x57905 sparse matrix of type '<class 'numpy.float64'>'
	with 7495000 stored elements in Compressed Sparse Row format>

In [23]:
from pathlib import Path
from scipy.io import mmread, mmwrite
from scipy.sparse import csr_matrix


def correct_matrix_function(foldername):
    # first correct the barcodes so that only the barcodes remain
    folder = Path(foldername)
    # print(folder)
    A = mmread(folder / "matrix.mtx.gz")
    A = csr_matrix(A).T
    mmwrite(folder / "matrix2", A)
    os.system("gzip " + str(folder / "matrix2.mtx"))

In [ ]:
def correct_barcodes_function(foldername):
    # first correct the barcodes so that only the barcodes remain
    folder = Path(foldername)
    # print(folder)
    barcode_file = folder / "barcodes.tsv.gz"  # str(folder + "/barcodes.tsv.gz")
    print(barcode_file)
    bc = pd.read_csv(barcode_file, sep="\t")
    try:
        bc.drop(["hg19_counts", "mm10_counts", "call"], axis=1, inplace=True)
        bc.to_csv(barcode_file, compression="gzip", header=False, index=False, sep="\t")
    except:
        pass

In [14]:
def correct_features_function(foldername):
    # first correct the barcodes so that only the barcodes remain
    folder = Path(foldername)
    # print(folder)
    features_file = folder / "features.tsv.gz"
    print(features_file)
    bc = pd.read_csv(features_file, sep="\t")
    bc.to_csv(features_file, compression="gzip", header=False, index=False, sep="\t")

In [38]:
for file in files:
    correct_barcodes_function(file)

GSM4795825_REH_TK/barcodes.tsv.gz
GSM4795824_REH_DMSO/barcodes.tsv.gz
GSM4795826_REH_XR/barcodes.tsv.gz


In [15]:
for file in files:
    correct_features_function(file)

GSM4795825_REH_TK/features.tsv.gz
GSM4795824_REH_DMSO/features.tsv.gz
GSM4795826_REH_XR/features.tsv.gz


In [24]:
for file in files:
    correct_matrix_function(file)

In [25]:

import scanpy as sc
folder=files[0]
adata=sc.read_10x_mtx(str(folder))
adata

AnnData object with n_obs × n_vars = 2144 × 57905
    var: 'gene_ids', 'feature_types'

In [26]:
folder=files[1]
adata=sc.read_10x_mtx(str(folder))
adata

AnnData object with n_obs × n_vars = 914 × 57905
    var: 'gene_ids', 'feature_types'

In [27]:
folder=files[2]
adata=sc.read_10x_mtx(str(folder))
adata

AnnData object with n_obs × n_vars = 2466 × 57905
    var: 'gene_ids', 'feature_types'

In [3]:

features = pd.read_csv('/Users/Ali/Desktop/Caron et al./features.tsv.gz', header=None, sep='\t')
features[2] = "Gene Expression"
features.to_csv("/Users/Ali/Desktop/Caron et al./features.tsv.gz", header=False, index=False , compression = 'gzip', sep = '\t')


In [5]:
import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

adata = sc.read_10x_mtx(
    '/Users/Ali/Desktop/Caron et al.',
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)  


scanpy==1.9.3 anndata==0.9.1 umap==0.5.3 numpy==1.23.5 scipy==1.11.1 pandas==1.5.3 scikit-learn==1.3.0 statsmodels==0.14.0 python-igraph==0.10.5 pynndescent==0.5.10
... reading from cache file cache/Users-Ali-Desktop-Caron et al.-matrix.h5ad
